In [12]:
import sys
import os

# Añadir la carpeta raíz del proyecto al sys.path
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(PROJECT_ROOT)

from langchain_community.vectorstores import FAISS
from src.get_embedding_function import get_embedding_function

In [13]:
# Configurar ruta del índice FAISS
PROJECT_ROOT = os.path.abspath(os.path.join(os.getcwd(), ".."))
FAISS_INDEX_DIR = os.path.join(PROJECT_ROOT, "data")
print(f"📂 Directorio de índices FAISS: {FAISS_INDEX_DIR}")

# Función para obtener la ruta del índice según el modelo y proveedor
def get_faiss_path(provider, model_name):
    safe_model_name = model_name.replace("/", "_").replace(":", "_")
    return os.path.join(FAISS_INDEX_DIR, f"faiss_index_{provider}_{safe_model_name}")

📂 Directorio de índices FAISS: c:\Users\zepol\Documents\UAM\TFM\extranjerIA\data


In [14]:
# Configurar el modelo y proveedor
embedding_provider = "huggingface"
embedding_model = "distiluse-base-multilingual-cased-v1"

# Obtener la ruta del índice FAISS
faiss_path = get_faiss_path(embedding_provider, embedding_model)

# Verificar si el índice existe
if not os.path.exists(faiss_path):
    raise FileNotFoundError(f"Índice FAISS no encontrado en: {faiss_path}")

print(f"📂 Cargando índice FAISS desde: {faiss_path}")

# Cargar los embeddings
embeddings = get_embedding_function(embedding_model, embedding_provider)

# Cargar el índice FAISS
faiss_index = FAISS.load_local(faiss_path, embeddings, allow_dangerous_deserialization=True)
print("✅ Índice FAISS cargado correctamente.")

📂 Cargando índice FAISS desde: c:\Users\zepol\Documents\UAM\TFM\extranjerIA\data\faiss_index_huggingface_distiluse-base-multilingual-cased-v1
⚙️ Cargando modelo de embeddings de Hugging Face: distiluse-base-multilingual-cased-v1
✅ Índice FAISS cargado correctamente.


In [16]:
# Obtener el número de documentos en el índice
num_docs = len(faiss_index.index_to_docstore_id)
print(f"📊 Número de documentos en el índice: {num_docs}")

📊 Número de documentos en el índice: 511


In [17]:
# Recuperar un documento aleatorio del índice
doc_id = list(faiss_index.index_to_docstore_id.keys())[0]
doc = faiss_index.docstore.search(doc_id)

print("📄 Documento recuperado:")
print(doc)

📄 Documento recuperado:
ID 0 not found.


In [20]:
# Realizar una búsqueda de prueba
query_text = "¿Cuáles son los requisitos para la residencia en España?"

results = faiss_index.similarity_search_with_score(query_text, k=3)

print(f"🔍 Número de resultados obtenidos: {len(results)}")

# Inspeccionar los resultados
for i, result in enumerate(results):
    print(f"\nResultado {i + 1}:")
    print(result)
    print(f"Tipo del resultado: {type(result)}")
    print(f"Número de elementos: {len(result)}")

🔍 Número de resultados obtenidos: 3

Resultado 1:
(Document(id='05d9e3ed-0668-4aed-9c40-a97fdf981e00', metadata={'producer': 'Microsoft® Word LTSC', 'creator': 'Microsoft® Word LTSC', 'creationdate': '2024-04-12T11:24:27+02:00', 'source': 'c:\\Users\\zepol\\Documents\\UAM\\TFM\\extranjerIA\\data\\pdfs\\Indice_Documentacion_Esencial.pdf0.pdf', 'file_path': 'c:\\Users\\zepol\\Documents\\UAM\\TFM\\extranjerIA\\data\\pdfs\\Indice_Documentacion_Esencial.pdf0.pdf', 'total_pages': 8, 'format': 'PDF 1.7', 'title': '', 'author': 'ANA ESTER CEDILLO FREY', 'subject': '', 'keywords': '', 'moddate': '2024-04-12T11:24:27+02:00', 'trapped': '', 'page': 1}, page_content='Migraciones (inclusion.gob.es) \n• Padrón histórico, etc. (permanencia continuada en España durante los 2 años anteriores a la solicitud). \n• Compromiso de formación EX10. \nLABORAL. Más información en: Autorización de residencia temporal por circunstancias excepcionales. \nArraigo laboral (HI 35) - Migraciones - Ministerio de Inclus

In [19]:
# Verificar el contenido del docstore
print("📊 Contenido del docstore:")
print(faiss_index.docstore._dict)

📊 Contenido del docstore:
{'31fc5c89-c27e-4d7e-962c-8e44b787248b': Document(id='31fc5c89-c27e-4d7e-962c-8e44b787248b', metadata={'producer': 'SAMBox 2.1.16', 'creator': 'PDFsam Basic v4.1.3', 'creationdate': '', 'source': 'c:\\Users\\zepol\\Documents\\UAM\\TFM\\extranjerIA\\data\\pdfs\\16Familiar_comunitario_inicial.pdf.pdf', 'file_path': 'c:\\Users\\zepol\\Documents\\UAM\\TFM\\extranjerIA\\data\\pdfs\\16Familiar_comunitario_inicial.pdf.pdf', 'total_pages': 5, 'format': 'PDF 1.5', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'moddate': '2022-03-03T11:55:17+01:00', 'trapped': '', 'page': 0}, page_content='Actualizada a 2 de marzo de 2022 \n \n \n \n \n \n \n\uf0d8  Ciudadanos a los que es de aplicación \n \nLos familiares de ciudadano español o de otro Estado miembro de la Unión Europea, o de otro Estado \nparte en el Acuerdo sobre el Espacio Económico Europeo o Suiza, que no ostenten la nacionalidad de \nuno de dichos Estados, cuando se reúnan con él o le acompañen, y vaya

In [18]:
# Verificar si hay vectores en el índice FAISS
if faiss_index.index.ntotal == 0:
    print("❌ El índice FAISS no contiene vectores.")
else:
    print(f"✅ El índice FAISS contiene {faiss_index.index.ntotal} vectores.")

✅ El índice FAISS contiene 511 vectores.
